# Personal Finance Managment Tool

In [1]:
# Import libraries
import pandas as pd
from pathlib import Path
import csv
import numpy as np
import panel as pn
import datetime
from datetime import datetime, timezone
import hvplot.pandas

In [2]:
 # Import csv file
filepath = Path("../Personal-Finance-Management-Tool_Project1/Data/personal-transaction.csv")

In [3]:
# Read the csv as a dataframe 
df= pd.read_csv(filepath, index_col='Date', infer_datetime_format=True, parse_dates=True)
df.head()

/var/folders/r4/jh2rbs2n3js2641fckxtvh_c0000gn/T/ipykernel_18403/3313459864.py:2: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df= pd.read_csv(filepath, index_col='Date', infer_datetime_format=True, parse_dates=True)


,Month,Description,Amount,Category,Account Name
Date,,,,,
2018-01-01,JANUARY,Amazon,11.11,Shopping,Platinum Card
2018-01-02,JANUARY,Mortgage Payment,1247.44,Mortgage & Rent,Checking
2018-01-02,JANUARY,Restaurant,24.22,Food,Silver Card
2018-01-05,JANUARY,Restaurant,25.85,Food,Silver Card
2018-01-08,JANUARY,Gas Company,45.00,Utilities,Checking


# Clean Up data 
### Retrieve data types to confirm what needs to be converted
### Drop the extra date column

In [4]:
# Retrieve data types to confirm what needs to be converted
df.dtypes

Month            object
Description      object
Amount          float64
Category         object
Account Name     object
dtype: object

In [5]:
df.count()

Month           270
Description     270
Amount          270
Category        270
Account Name    270
dtype: int64

In [6]:
# Drop the extra date column
df.drop(columns=['Account Name'], inplace=True)
df.head()

,Month,Description,Amount,Category
Date,,,,
2018-01-01,JANUARY,Amazon,11.11,Shopping
2018-01-02,JANUARY,Mortgage Payment,1247.44,Mortgage & Rent
2018-01-02,JANUARY,Restaurant,24.22,Food
2018-01-05,JANUARY,Restaurant,25.85,Food
2018-01-08,JANUARY,Gas Company,45.00,Utilities


In [7]:
# Count the frequency of each sector from the list of companies
df_count= df['Description'].value_counts()
df_count

Description
Restaurant            81
Grocery Store         71
Amazon                37
Shell                 33
Mortgage Payment      12
Gas Company           12
City Water Charges    12
Power Company         12
Name: count, dtype: int64

# Group Categories
### Filter by category and adding all the amounts by month


In [8]:
df['Category'] = 'unassigned'
df.head()

,Month,Description,Amount,Category
Date,,,,
2018-01-01,JANUARY,Amazon,11.11,unassigned
2018-01-02,JANUARY,Mortgage Payment,1247.44,unassigned
2018-01-02,JANUARY,Restaurant,24.22,unassigned
2018-01-05,JANUARY,Restaurant,25.85,unassigned
2018-01-08,JANUARY,Gas Company,45.00,unassigned


In [9]:
# Assign transactions to the correct category
# Shopping
df['Category'] = np.where(df['Description'].str.contains(
    'Amazon|Grocery Store'),
    'Shopping', df['Category'])

# Mortgage Rent
df['Category'] = np.where(df['Description'].str.contains(
    'Mortgage Payment'),
    'Mortgage Rent', df['Category'])

# Food
df['Category'] = np.where(df['Description'].str.contains(
    'Restaurant'),
    'Food', df['Category'])

# Utilities
df['Category'] = np.where(df['Description'].str.contains(
    'Gas Company|City Water Charges|Power Company'),
    'Utilities', df['Category'])

# Gas & Fuel
df['Category'] = np.where(df['Description'].str.contains(
    'Shell'),
    'Gas & Fuel', df['Category'])
df.head(15)


,Month,Description,Amount,Category
Date,,,,
2018-01-01,JANUARY,Amazon,11.11,Shopping
2018-01-02,JANUARY,Mortgage Payment,1247.44,Mortgage Rent
2018-01-02,JANUARY,Restaurant,24.22,Food
2018-01-05,JANUARY,Restaurant,25.85,Food
2018-01-08,JANUARY,Gas Company,45.00,Utilities
2018-01-11,JANUARY,Shell,34.87,Gas & Fuel
2018-01-11,JANUARY,Grocery Store,43.54,Shopping
2018-01-13,JANUARY,Restaurant,32.91,Food
2018-01-13,JANUARY,Amazon,39.05,Shopping


In [10]:
# Group dataframe by consisting of month, amount and category
df2 = df[['Month','Amount','Category']]
df2

,Month,Amount,Category
Date,,,
2018-01-01,JANUARY,11.11,Shopping
2018-01-02,JANUARY,1247.44,Mortgage Rent
2018-01-02,JANUARY,24.22,Food
2018-01-05,JANUARY,25.85,Food
2018-01-08,JANUARY,45.00,Utilities
...,...,...,...
2018-12-26,DECEMBER,7.48,Gas & Fuel
2018-12-26,DECEMBER,44.99,Shopping
2018-12-28,DECEMBER,12.00,Food


In [11]:
# Group dataframe by category and sum amount
df3 = df2.groupby('Category').Amount.sum()
df3

Category
Food              2107.98
Gas & Fuel        1095.91
Mortgage Rent    14854.50
Shopping          3241.66
Utilities         1582.00
Name: Amount, dtype: float64

In [12]:
# Group dataframe by month and sum amount
df4 = df2.groupby('Month').Amount.sum()
df4

Month
APRIL        1985.13
AUGUST       1730.84
DECEMBER     2254.32
FEBURARY     1858.48
JANUARY      1833.54
JULY         1944.19
JUNE         1819.15
MARCH        1972.20
MAY          2108.93
NOVEMBER     1683.27
OCTOBER      1810.73
SEPTEMBER    1881.27
Name: Amount, dtype: float64

In [13]:
# Plot bar chart of last year's expenses
# Specify the variables for the x- and y-axes using the syntax
df_plot = df3.hvplot(kind ='bar', 
           x='Category', 
           y='Amount') \
   .opts(title="Last Year's Expenses ", 
           xlabel='Category', 
           ylabel='Amount', 
           yformatter='%.0f')
df_plot

:Bars   [Category]   (Amount)

# Summary of Last Month's Income


In [18]:
# Get the latest month and year 
latest_month = df2['Month'].max()
latest_month


'SEPTEMBER'

In [15]:
# Filter the dataframe to include only transactions from the latest month
latest_month_expenses = df[(df['Month']== latest_month)] 

In [16]:
latest_month_expenses = latest_month_expenses.groupby('Category')['Amount'].sum().reset_index()

latest_month_expenses['Amount'] = latest_month_expenses['Amount'].astype('str')
latest_month_expenses['Amount'] = latest_month_expenses['Amount'].str.replace('-','')
# Get absolute figure
latest_month_expenses['Amount'] = latest_month_expenses['Amount'].astype('float')

latest_month_expenses= latest_month_expenses[latest_month_expenses["Category"].str.contains("Excluded|unassigned") == False]
# Sort value
latest_month_expenses = latest_month_expenses.sort_values(by='Amount', ascending=False)
# Round value 
latest_month_expenses['Amount'] = latest_month_expenses['Amount'].round().astype(int)

latest_month_expenses

,Category,Amount
2,Mortgage Rent,1247
3,Shopping,268
4,Utilities,125
0,Food,124
1,Gas & Fuel,118


In [17]:
# Total amount of all categories
latest_month_expenses_tot = latest_month_expenses['Amount'].sum()
latest_month_expenses_tot

1882

# Confirming if person spent right the whole year

# What new yearly goal they would possibly like next time

# Chart 